In [98]:
import os
import SimpleITK as sitk
import pandas as pd
import numpy as np
from tqdm import tqdm
import glob

In [99]:
root = "/media/yesindeed/DATADRIVE1/mount/remote_cse/datasets/multitask-moe/luna16"

In [100]:
df_candidates = pd.read_csv(os.path.join(
    root, "candidates_V2/candidates_V2.csv"))


df_annotation = pd.read_csv(os.path.join(root, "annotations.csv"))

In [101]:
df_annotation

,seriesuid,coordX,coordY,coordZ,diameter_mm
0,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,-128.699421,-175.319272,-298.387506,5.651471
1,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,103.783651,-211.925149,-227.121250,4.224708
2,1.3.6.1.4.1.14519.5.2.1.6279.6001.100398138793...,69.639017,-140.944586,876.374496,5.786348
3,1.3.6.1.4.1.14519.5.2.1.6279.6001.100621383016...,-24.013824,192.102405,-391.081276,8.143262
4,1.3.6.1.4.1.14519.5.2.1.6279.6001.100621383016...,2.441547,172.464881,-405.493732,18.545150
...,...,...,...,...,...
1181,1.3.6.1.4.1.14519.5.2.1.6279.6001.994459772950...,-160.856298,-28.560349,-269.168728,5.053694
1182,1.3.6.1.4.1.14519.5.2.1.6279.6001.994459772950...,-102.189570,-73.865766,-220.536241,4.556101
1183,1.3.6.1.4.1.14519.5.2.1.6279.6001.994459772950...,-37.535409,64.041949,-127.687101,4.357368
1184,1.3.6.1.4.1.14519.5.2.1.6279.6001.997611074084...,43.196112,74.438486,-200.523314,4.277203


In [102]:
df_candidates["class"].value_counts()

class
0    753418
1      1557
Name: count, dtype: int64

In [103]:
path_dict = {"seriesuid": [], "path": []}

for file in glob.glob(f"{root}/subset*/subset*/*.mhd"):
    path_dict["seriesuid"].append(os.path.basename(file)[:-4])
    path_dict["path"].append(file.replace(root, "."))

df_path = pd.DataFrame.from_dict(path_dict)

In [104]:
df_path

,seriesuid,path
0,1.3.6.1.4.1.14519.5.2.1.6279.6001.105756658031...,./subset0/subset0/1.3.6.1.4.1.14519.5.2.1.6279...
1,1.3.6.1.4.1.14519.5.2.1.6279.6001.108197895896...,./subset0/subset0/1.3.6.1.4.1.14519.5.2.1.6279...
2,1.3.6.1.4.1.14519.5.2.1.6279.6001.109002525524...,./subset0/subset0/1.3.6.1.4.1.14519.5.2.1.6279...
3,1.3.6.1.4.1.14519.5.2.1.6279.6001.111172165674...,./subset0/subset0/1.3.6.1.4.1.14519.5.2.1.6279...
4,1.3.6.1.4.1.14519.5.2.1.6279.6001.122763913896...,./subset0/subset0/1.3.6.1.4.1.14519.5.2.1.6279...
...,...,...
440,1.3.6.1.4.1.14519.5.2.1.6279.6001.811825890493...,./subset4/subset4/1.3.6.1.4.1.14519.5.2.1.6279...
441,1.3.6.1.4.1.14519.5.2.1.6279.6001.826829446346...,./subset4/subset4/1.3.6.1.4.1.14519.5.2.1.6279...
442,1.3.6.1.4.1.14519.5.2.1.6279.6001.885168397833...,./subset4/subset4/1.3.6.1.4.1.14519.5.2.1.6279...
443,1.3.6.1.4.1.14519.5.2.1.6279.6001.897684031374...,./subset4/subset4/1.3.6.1.4.1.14519.5.2.1.6279...


In [105]:
df_path["seriesuid"].tolist()[0]

'1.3.6.1.4.1.14519.5.2.1.6279.6001.105756658031515062000744821260'

In [106]:
df_candidates["seriesuid"].tolist()[0]

'1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222365663678666836860'

In [107]:
contains_annotation = [(x in df_annotation["seriesuid"].tolist())
                       for x in df_path["seriesuid"].tolist()]

In [108]:
np.sum(contains_annotation)

311

In [109]:
len(contains_annotation)

445

In [110]:
def xyz2irc(xyz_coord, xyz_origin, xyz_sizes, irc_transform_mat):
    """
    Map the patient coordidates to the voxel-based coordinates through data included in the meta-data file of CT.

    Parameters:
        - xyz_coord (XYZ_tuple): the patient coordinates to be transformed.
        - xyz_origin (XYZ_tuple): the exact origin point in the patient space for reference.
        - xyz_sizes (XYZ_tuple): the voxel size for scaling purposes.
        - irc_transform_mat (np.array): the transformation matrix between the two spaces.

    return:
        transformed coordinate as IRC_tuple
    """
    coordinate_xyz = np.array(xyz_coord)
    physical_origin = np.array(xyz_origin)
    physical_sizes = np.array(xyz_sizes)
    # reverse the computations
    cri_coord = ((coordinate_xyz - physical_origin) @ np.linalg.inv(irc_transform_mat)) / physical_sizes
    rounded_cri_coord = np.round(cri_coord).astype(int)
    return rounded_cri_coord[::-1]

In [111]:
for i in range(len(df_annotation)):
    item = df_annotation.iloc[i]

    series_id, x, y, z, diameter = item

    if not series_id in df_path["seriesuid"].tolist():
        continue

    path = df_path.loc[df_path["seriesuid"] == series_id]["path"].tolist()[0]

    print(f"path: {path}")

    ct_mhd = sitk.ReadImage(os.path.join(root, path))
    ct_arr = np.array(sitk.GetArrayFromImage(ct_mhd),
                      dtype=np.float32).clip(-600, 1500)

    origin = ct_mhd.GetOrigin()
    spacing = ct_mhd.GetSpacing()
    transform_mat = np.array(ct_mhd.GetDirection()).reshape(3, 3)

    irc = xyz2irc((x,  y, z), origin, spacing, transform_mat)

    print(origin)
    print(spacing)
    break

path: ./subset2/subset2/1.3.6.1.4.1.14519.5.2.1.6279.6001.100621383016233746780170740405.mhd
(-120.049467, 9.48041606, -657.0)
(0.556640625, 0.556640625, 1.0)


In [112]:
irc

array([266, 328, 173])

In [113]:
diameter

8.143261683

In [114]:
seg_file = os.path.join(root, "seg-lungs-LUNA16",
                        "seg-lungs-LUNA16", f"{series_id}.mhd")

seg = np.array(sitk.GetArrayFromImage(sitk.ReadImage(seg_file)))

In [116]:
seg.shape

(321, 512, 512)